In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/TFG-AnalisisResultados'

/content/drive/MyDrive/TFG-AnalisisResultados


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# LD

In [4]:
pd.set_option('display.max_columns', None)
sgp = pd.read_csv('./LD_SGP_acquisition_summary.csv')
sgp

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,16.2,4.969909,8,21,22.577237,13.319037,14.093970,45.950889,136.223799,3.058388,133.400483,139.540562,134.174263,3.413101,3.041130,1.358000,128.742536,136.223799,15.083624,1.885289,1.341123,0.442973,250.0366,100.294373,107.954,383.292
1,qParEGO,12.0,4.743416,6,19,18.373729,24.410911,0.829160,60.339777,137.611874,2.487627,134.166199,140.819905,137.338876,2.531955,0.325236,0.225333,136.712755,137.611874,17.243604,3.182096,6.318467,1.103243,215.3916,105.518697,77.033,353.333
2,qQEHVI,17.0,9.273618,6,31,12.992811,7.768957,2.487435,21.583325,137.064674,2.532384,133.158035,139.609437,136.681436,2.670978,0.278735,0.314235,136.332187,137.064674,19.674458,3.973130,6.070232,1.634884,362.1582,252.688805,81.539,759.882
3,qQNEHVI,12.4,5.549775,8,22,17.099714,15.386897,1.247219,37.859386,137.000070,2.753605,133.686481,139.494449,136.772574,2.764798,0.298314,0.106241,136.240098,137.000070,19.062879,4.074858,5.406538,0.674191,243.0520,141.513090,136.266,480.657


In [5]:
# Round all numeric values to 2 decimal places for presentation
df = sgp.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV          | Pareto Spacing   | Mean HV Over Time   | Runtime (s)   |
|---:|:-----------------------|:-------------|:------------------|:-----------------|:--------------------|:--------------|
|  0 | Sobol                  | 16.2 ± 4.97  | 136.22 ± 3.06     | 22.58 ± 13.32    | 134.17 ± 3.41       | 15.08 ± 1.89  |
|  1 | qParEGO                | 12.0 ± 4.74  | **137.61 ± 2.49** | 18.37 ± 24.41    | 137.34 ± 2.53       | 17.24 ± 3.18  |
|  2 | qQEHVI                 | 17.0 ± 9.27  | 137.06 ± 2.53     | 12.99 ± 7.77     | 136.68 ± 2.67       | 19.67 ± 3.97  |
|  3 | qQNEHVI                | 12.4 ± 5.55  | 137.0 ± 2.75      | 17.1 ± 15.39     | 136.77 ± 2.76       | 19.06 ± 4.07  |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 16.2 ± 4.97 & 136.22 ± 3.06 & 22.58 ± 13.32 & 134.17 ± 3.41 & 15.08 ± 1.89 \\
qParEGO & 12.0 ± 4.74 & **137

In [6]:
mgp = pd.read_csv('./LD_MGP_acquisition_summary.csv')
mgp

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,16.2,4.969909,8,21,22.577237,13.319037,14.093970,45.950889,136.223799,3.058388,133.400483,139.540562,134.174263,3.413101,3.041130,1.358000,128.742536,136.223799,14.795169,0.626992,1.283555,0.408849,238.5076,69.692637,120.672,291.280
1,qParEGO,15.8,4.816638,12,24,10.953168,21.291562,0.000000,48.946497,137.826677,3.112932,133.203695,141.621855,136.859965,2.380413,1.325849,1.560870,133.886797,137.826677,21.105733,2.758802,5.475041,1.197769,330.4708,92.654447,252.210,459.730
2,qQEHVI,19.2,6.220932,15,30,4.855575,3.189590,1.658312,9.391485,137.304671,3.173922,133.247313,140.350772,136.779726,3.005689,0.511896,0.201058,136.025376,137.304671,22.637623,5.377976,7.020772,0.978437,435.2768,166.383298,252.747,662.883
3,qQNEHVI,17.8,8.348653,11,32,21.542521,8.902357,10.732543,32.155228,137.049908,2.895406,133.080405,140.072219,136.739585,2.846654,0.368771,0.151438,135.997066,137.049908,21.193775,2.127916,6.377465,0.895974,383.9888,198.334353,191.847,713.007


In [7]:
# Round all numeric values to 2 decimal places for presentation
df = mgp.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV          | Pareto Spacing   | Mean HV Over Time   | Runtime (s)   |
|---:|:-----------------------|:-------------|:------------------|:-----------------|:--------------------|:--------------|
|  0 | Sobol                  | 16.2 ± 4.97  | 136.22 ± 3.06     | 22.58 ± 13.32    | 134.17 ± 3.41       | 14.8 ± 0.63   |
|  1 | qParEGO                | 15.8 ± 4.82  | **137.83 ± 3.11** | 10.95 ± 21.29    | 136.86 ± 2.38       | 21.11 ± 2.76  |
|  2 | qQEHVI                 | 19.2 ± 6.22  | 137.3 ± 3.17      | 4.86 ± 3.19      | 136.78 ± 3.01       | 22.64 ± 5.38  |
|  3 | qQNEHVI                | 17.8 ± 8.35  | 137.05 ± 2.9      | 21.54 ± 8.9      | 136.74 ± 2.85       | 21.19 ± 2.13  |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 16.2 ± 4.97 & 136.22 ± 3.06 & 22.58 ± 13.32 & 134.17 ± 3.41 & 14.8 ± 0.63 \\
qParEGO & 15.8 ± 4.82 & **137.

In [8]:
mgp1 = pd.read_csv('./LD_MGPR1_acquisition_summary.csv')
mgp1

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,16.2,4.969909,8,21,22.577237,13.319037,14.093970,45.950889,136.223799,3.058388,133.400483,139.540562,134.174263,3.413101,3.041130,1.358000,128.742536,136.223799,15.231040,2.140930,1.321552,0.341913,248.1924,82.169988,108.614,319.772
1,qParEGO,10.4,2.302173,8,14,12.414136,8.456425,0.471404,24.041609,137.233272,2.923340,133.425074,140.322698,136.516427,2.918575,1.416899,2.356607,133.712180,137.233272,19.957684,1.193345,5.810661,1.191422,208.4728,52.953394,160.819,289.459
2,qQEHVI,13.8,4.381780,9,17,20.278483,17.615632,1.000018,39.335098,136.950289,2.504705,133.604476,139.566617,136.183656,2.323584,1.118017,1.550490,133.792692,136.950289,22.084482,3.652946,5.581979,1.314068,297.2634,81.705592,194.140,404.306
3,qQNEHVI,16.0,14.053469,8,41,26.190703,16.101168,8.697943,51.777985,136.866626,2.523283,133.142194,139.220693,136.645615,2.651229,0.248277,0.225484,136.157400,136.866626,18.893887,2.162497,5.125518,0.925731,325.8758,338.838144,142.353,930.353


In [9]:
# Round all numeric values to 2 decimal places for presentation
df = mgp1.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV          | Pareto Spacing   | Mean HV Over Time   | Runtime (s)   |
|---:|:-----------------------|:-------------|:------------------|:-----------------|:--------------------|:--------------|
|  0 | Sobol                  | 16.2 ± 4.97  | 136.22 ± 3.06     | 22.58 ± 13.32    | 134.17 ± 3.41       | 15.23 ± 2.14  |
|  1 | qParEGO                | 10.4 ± 2.3   | **137.23 ± 2.92** | 12.41 ± 8.46     | 136.52 ± 2.92       | 19.96 ± 1.19  |
|  2 | qQEHVI                 | 13.8 ± 4.38  | 136.95 ± 2.5      | 20.28 ± 17.62    | 136.18 ± 2.32       | 22.08 ± 3.65  |
|  3 | qQNEHVI                | 16.0 ± 14.05 | 136.87 ± 2.52     | 26.19 ± 16.1     | 136.65 ± 2.65       | 18.89 ± 2.16  |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 16.2 ± 4.97 & 136.22 ± 3.06 & 22.58 ± 13.32 & 134.17 ± 3.41 & 15.23 ± 2.14 \\
qParEGO & 10.4 ± 2.3 & **137.

# LD NEURO

In [10]:
pd.set_option('display.max_columns', None)
sgp = pd.read_csv('./LD_NEURO_SGP_acquisition_summary.csv')
sgp

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,22.0,7.449832,15,34,15.480454,15.241079,4.974912,40.466020,134.664050,1.617690,132.611422,136.841833,132.733013,2.771306,2.779786,1.330527,126.684824,134.664050,15.002447,1.968625,1.230070,0.409994,331.2880,117.911687,197.924,486.495
1,qParEGO,25.2,7.224957,15,35,15.017138,13.182241,1.277736,32.785400,136.120654,2.025622,133.652729,139.181081,135.638096,1.956598,0.624945,0.279714,134.002335,136.120654,30.127808,4.504868,8.775523,3.553922,782.6000,328.366617,369.636,1280.536
2,qQEHVI,14.2,5.403702,8,21,11.084139,7.577822,3.269177,22.504901,135.304112,1.460026,133.547059,137.395381,134.745762,1.383888,0.590203,0.356483,133.820296,135.304112,21.200178,3.907529,5.608077,1.236101,291.6188,97.294628,192.961,441.045
3,qQNEHVI,27.0,8.746428,18,40,17.378406,22.627575,1.833004,57.170732,136.106616,2.038630,132.937655,138.614374,135.193256,1.566705,0.807530,0.509950,133.799364,136.106616,23.992230,1.794907,6.048567,0.555055,652.3080,234.754965,395.357,1021.289


In [11]:
# Round all numeric values to 2 decimal places for presentation
df = sgp.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV          | Pareto Spacing   | Mean HV Over Time   | Runtime (s)   |
|---:|:-----------------------|:-------------|:------------------|:-----------------|:--------------------|:--------------|
|  0 | Sobol                  | 22.0 ± 7.45  | 134.66 ± 1.62     | 15.48 ± 15.24    | 132.73 ± 2.77       | 15.0 ± 1.97   |
|  1 | qParEGO                | 25.2 ± 7.22  | **136.12 ± 2.03** | 15.02 ± 13.18    | 135.64 ± 1.96       | 30.13 ± 4.5   |
|  2 | qQEHVI                 | 14.2 ± 5.4   | 135.3 ± 1.46      | 11.08 ± 7.58     | 134.75 ± 1.38       | 21.2 ± 3.91   |
|  3 | qQNEHVI                | 27.0 ± 8.75  | 136.11 ± 2.04     | 17.38 ± 22.63    | 135.19 ± 1.57       | 23.99 ± 1.79  |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 22.0 ± 7.45 & 134.66 ± 1.62 & 15.48 ± 15.24 & 132.73 ± 2.77 & 15.0 ± 1.97 \\
qParEGO & 25.2 ± 7.22 & **136.

In [12]:
mgp = pd.read_csv('./LD_NEURO_MGP_acquisition_summary.csv')
mgp

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,22.0,7.449832,15,34,15.480454,15.241079,4.974912,40.466020,134.664050,1.617690,132.611422,136.841833,132.733013,2.771306,2.779786,1.330527,126.684824,134.664050,14.128806,1.879589,1.171453,0.328654,308.8248,98.644164,197.620,449.551
1,qParEGO,20.4,9.476286,10,35,7.673011,6.581934,1.490707,17.976053,136.048152,1.702058,134.415477,138.242942,134.882369,1.736989,1.506605,1.749154,131.814237,136.048152,21.167612,5.656891,5.779450,0.526018,431.6174,206.390212,159.047,726.218
2,qQEHVI,16.6,7.300685,10,28,26.788296,13.864560,8.454115,39.637578,135.848938,0.916077,134.331855,136.746542,135.384497,0.895518,0.667208,0.487893,133.850978,135.848938,20.431158,2.744142,7.996347,6.686509,343.0302,179.934321,202.987,650.918
3,qQNEHVI,22.4,4.037326,18,27,16.970688,10.251602,5.899092,27.892625,136.162295,1.202182,134.904035,138.128080,135.482405,1.081358,0.742674,0.257108,133.771098,136.162295,21.996053,4.167404,6.237595,1.015009,499.5294,165.212086,371.326,739.479


In [13]:
# Round all numeric values to 2 decimal places for presentation
df = mgp.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV         | Pareto Spacing   | Mean HV Over Time   | Runtime (s)   |
|---:|:-----------------------|:-------------|:-----------------|:-----------------|:--------------------|:--------------|
|  0 | Sobol                  | 22.0 ± 7.45  | 134.66 ± 1.62    | 15.48 ± 15.24    | 132.73 ± 2.77       | 14.13 ± 1.88  |
|  1 | qParEGO                | 20.4 ± 9.48  | 136.05 ± 1.7     | 7.67 ± 6.58      | 134.88 ± 1.74       | 21.17 ± 5.66  |
|  2 | qQEHVI                 | 16.6 ± 7.3   | 135.85 ± 0.92    | 26.79 ± 13.86    | 135.38 ± 0.9        | 20.43 ± 2.74  |
|  3 | qQNEHVI                | 22.4 ± 4.04  | **136.16 ± 1.2** | 16.97 ± 10.25    | 135.48 ± 1.08       | 22.0 ± 4.17   |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 22.0 ± 7.45 & 134.66 ± 1.62 & 15.48 ± 15.24 & 132.73 ± 2.77 & 14.13 ± 1.88 \\
qParEGO & 20.4 ± 9.48 & 136.05 ± 1.

In [14]:
mgp1 = pd.read_csv('./LD_NEURO_MGPR1_acquisition_summary.csv')
mgp1

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,22.0,7.449832,15,34,15.480454,15.241079,4.974912,40.466020,134.664050,1.617690,132.611422,136.841833,132.733013,2.771306,2.779786,1.330527,126.684824,134.664050,13.673590,0.618242,1.132342,0.379892,300.8448,102.089661,211.110,457.598
1,qParEGO,17.8,3.492850,16,24,12.072316,13.079131,3.269124,34.794765,136.021087,1.138912,134.616620,137.543127,134.622210,1.178455,2.084674,2.263818,129.677321,136.021087,20.801557,2.162474,6.285877,1.239081,374.7272,109.808238,285.641,561.872
2,qQEHVI,19.4,2.073644,17,22,21.494957,20.285287,0.000000,53.105927,136.217692,1.851175,134.453204,138.651324,135.216312,1.654209,1.353085,0.821517,131.577796,136.217692,20.393621,1.233873,5.488779,0.913969,394.5758,36.132080,359.901,436.652
3,qQNEHVI,16.4,4.505552,13,24,21.427530,13.590934,3.342674,37.006749,135.228253,1.988578,133.380704,138.037918,134.815179,1.835230,0.522422,0.297778,133.772163,135.228253,18.957429,2.515775,4.950435,1.367157,317.8192,131.834697,233.706,548.934


In [15]:
# Round all numeric values to 2 decimal places for presentation
df = mgp1.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV          | Pareto Spacing   | Mean HV Over Time   | Runtime (s)   |
|---:|:-----------------------|:-------------|:------------------|:-----------------|:--------------------|:--------------|
|  0 | Sobol                  | 22.0 ± 7.45  | 134.66 ± 1.62     | 15.48 ± 15.24    | 132.73 ± 2.77       | 13.67 ± 0.62  |
|  1 | qParEGO                | 17.8 ± 3.49  | 136.02 ± 1.14     | 12.07 ± 13.08    | 134.62 ± 1.18       | 20.8 ± 2.16   |
|  2 | qQEHVI                 | 19.4 ± 2.07  | **136.22 ± 1.85** | 21.49 ± 20.29    | 135.22 ± 1.65       | 20.39 ± 1.23  |
|  3 | qQNEHVI                | 16.4 ± 4.51  | 135.23 ± 1.99     | 21.43 ± 13.59    | 134.82 ± 1.84       | 18.96 ± 2.52  |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 22.0 ± 7.45 & 134.66 ± 1.62 & 15.48 ± 15.24 & 132.73 ± 2.77 & 13.67 ± 0.62 \\
qParEGO & 17.8 ± 3.49 & 136.0

# HD

In [16]:
pd.set_option('display.max_columns', None)
sgp = pd.read_csv('./HD_SGP_acquisition_summary.csv')
sgp

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,16.0,5.612486,8,22,57.603036,64.558373,0.000000,137.650120,429.376720,8.710619,413.817620,434.008368,422.388580,8.378762,10.272678,4.417553,403.945470,429.376720,177.757182,19.673297,15.573979,4.149286,2892.5350,1191.810892,1329.510,4232.235
1,qParEGO,17.2,9.391486,7,30,68.962572,88.636410,0.687195,221.856208,432.343224,5.276201,424.393389,437.425120,430.588949,3.740356,1.443625,1.362387,427.937874,432.343224,189.458320,72.226399,77.225298,21.410148,3644.4604,2667.400795,735.372,6481.309
2,qQEHVI,13.0,4.582576,6,18,22.991593,18.634723,0.000000,41.012185,430.699269,4.413330,423.916398,435.104941,429.029138,3.074212,1.447023,1.504745,427.424220,430.699269,138.375949,19.857312,55.237119,14.845057,1808.0462,710.157566,777.579,2717.938
3,qQNEHVI,16.2,9.959920,6,28,65.512169,85.330987,3.771233,200.921623,431.606339,2.903640,428.567344,435.659411,430.658955,2.891171,1.096993,1.021252,428.192215,431.606339,141.654809,44.732594,56.060279,22.453983,2538.7126,1828.519511,567.711,4484.354


In [17]:
# Round all numeric values to 2 decimal places for presentation
df = sgp.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV          | Pareto Spacing   | Mean HV Over Time   | Runtime (s)    |
|---:|:-----------------------|:-------------|:------------------|:-----------------|:--------------------|:---------------|
|  0 | Sobol                  | 16.0 ± 5.61  | 429.38 ± 8.71     | 57.6 ± 64.56     | 422.39 ± 8.38       | 177.76 ± 19.67 |
|  1 | qParEGO                | 17.2 ± 9.39  | **432.34 ± 5.28** | 68.96 ± 88.64    | 430.59 ± 3.74       | 189.46 ± 72.23 |
|  2 | qQEHVI                 | 13.0 ± 4.58  | 430.7 ± 4.41      | 22.99 ± 18.63    | 429.03 ± 3.07       | 138.38 ± 19.86 |
|  3 | qQNEHVI                | 16.2 ± 9.96  | 431.61 ± 2.9      | 65.51 ± 85.33    | 430.66 ± 2.89       | 141.65 ± 44.73 |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 16.0 ± 5.61 & 429.38 ± 8.71 & 57.6 ± 64.56 & 422.39 ± 8.38 & 177.76 ± 19.67 \\
qParEGO & 17.2 ± 9.39 

In [18]:
mgp = pd.read_csv('./HD_MGP_acquisition_summary.csv')
mgp

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,16.0,5.612486,8,22,57.603036,64.558373,0.000000,137.650120,429.376720,8.710619,413.817620,434.008368,422.388580,8.378762,10.272678,4.417553,403.945470,429.376720,174.599844,19.367625,15.829376,5.169584,2841.5744,1233.250673,1373.294,4570.509
1,qParEGO,20.8,10.353743,10,36,86.271807,68.288164,30.890487,188.211574,432.157173,4.644643,424.934038,436.680990,430.662899,3.745318,2.462110,2.584826,420.881982,432.157173,177.968785,23.071461,61.697599,12.790065,3695.8976,1755.986694,1579.004,5537.702
2,qQEHVI,15.2,7.661593,6,27,108.571158,62.382876,18.856175,174.790150,431.775970,4.648424,425.509430,438.240590,430.022921,3.077785,1.705366,1.629474,427.424015,431.775970,166.097727,25.949548,57.398463,10.294922,2537.9834,1326.252718,949.114,4427.509
3,qQNEHVI,24.2,10.401923,13,40,59.236690,33.463213,30.188727,96.956463,433.209480,3.969378,426.158707,435.717860,430.779499,3.202420,2.122095,0.653844,427.156283,433.209480,181.931887,30.882225,60.414623,12.576177,4401.4488,2004.829538,2421.174,6700.493


In [19]:
# Round all numeric values to 2 decimal places for presentation
df = mgp.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV          | Pareto Spacing   | Mean HV Over Time   | Runtime (s)    |
|---:|:-----------------------|:-------------|:------------------|:-----------------|:--------------------|:---------------|
|  0 | Sobol                  | 16.0 ± 5.61  | 429.38 ± 8.71     | 57.6 ± 64.56     | 422.39 ± 8.38       | 174.6 ± 19.37  |
|  1 | qParEGO                | 20.8 ± 10.35 | 432.16 ± 4.64     | 86.27 ± 68.29    | 430.66 ± 3.75       | 177.97 ± 23.07 |
|  2 | qQEHVI                 | 15.2 ± 7.66  | 431.78 ± 4.65     | 108.57 ± 62.38   | 430.02 ± 3.08       | 166.1 ± 25.95  |
|  3 | qQNEHVI                | 24.2 ± 10.4  | **433.21 ± 3.97** | 59.24 ± 33.46    | 430.78 ± 3.2        | 181.93 ± 30.88 |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 16.0 ± 5.61 & 429.38 ± 8.71 & 57.6 ± 64.56 & 422.39 ± 8.38 & 174.6 ± 19.37 \\
qParEGO & 20.8 ± 10.35 

In [20]:
mgp1 = pd.read_csv('./HD_MGPR1_acquisition_summary.csv')
mgp1

,acquisition,n_iterations_mean,n_iterations_std,n_iterations_min,n_iterations_max,pareto_spacing_mean,pareto_spacing_std,pareto_spacing_min,pareto_spacing_max,final_hv_mean,final_hv_std,final_hv_min,final_hv_max,hv_mean_mean,hv_mean_std,hv_std_mean,hv_std_std,hv_min_mean,hv_max_mean,time_mean_mean,time_mean_std,time_std_mean,time_std_std,cumulative_time_mean,cumulative_time_std,cumulative_time_min,cumulative_time_max
0,Sobol,16.0,5.612486,8,22,57.603036,64.558373,0.000000,137.650120,429.376720,8.710619,413.817620,434.008368,422.388580,8.378762,10.272678,4.417553,403.945470,429.376720,187.455111,22.593361,16.518328,2.910934,3038.1568,1142.939624,1244.729,4115.215
1,qParEGO,15.8,5.932959,10,24,100.408089,97.457562,3.999967,246.073156,430.472396,1.998192,428.475529,433.731982,426.744251,5.988669,6.017784,7.742388,413.844051,430.472396,159.769433,17.269947,61.895498,8.878244,2558.1988,1152.035701,1700.586,4354.163
2,qQEHVI,20.8,10.686440,7,31,60.294161,49.048076,4.686052,122.668133,432.396791,4.379435,427.196718,436.841974,429.597620,2.928802,3.400913,3.115454,420.000493,432.396791,175.730666,19.848872,55.983403,11.188420,3677.0518,2070.855795,1320.223,6254.750
3,qQNEHVI,20.2,6.978539,9,26,23.751241,8.772237,13.021349,35.391847,431.853740,4.063921,424.833771,435.312538,430.377536,3.421878,1.676746,0.748411,427.213164,431.853740,149.172138,5.962975,46.244823,10.442064,3004.1660,1033.665673,1373.325,3882.278


In [21]:
# Round all numeric values to 2 decimal places for presentation
df = mgp1.round(2)

# Create formatted columns with ± notation
df["Iterations"] = df["n_iterations_mean"].astype(str) + " ± " + df["n_iterations_std"].astype(str)
df["Final HV"] = df["final_hv_mean"].astype(str) + " ± " + df["final_hv_std"].astype(str)
df["Pareto Spacing"] = df["pareto_spacing_mean"].astype(str) + " ± " + df["pareto_spacing_std"].astype(str)
df["Mean HV Over Time"] = df["hv_mean_mean"].astype(str) + " ± " + df["hv_mean_std"].astype(str)
df["Runtime"] = df["time_mean_mean"].astype(str) + " ± " + df["time_mean_std"].astype(str)

# Keep only formatted columns
df_pretty = df[["acquisition", "Iterations", "Final HV", "Pareto Spacing", "Mean HV Over Time", "Runtime"]]

# Rename columns
df_pretty.columns = ["Acquisition Function", "Iterations", "Final HV",
                     "Pareto Spacing", "Mean HV Over Time", "Runtime (s)"]

# Optionally highlight best Final HV
best_idx = df["final_hv_mean"].idxmax()
df_pretty.loc[best_idx, "Final HV"] = "**" + df_pretty.loc[best_idx, "Final HV"] + "**"

print(df_pretty.to_markdown())
# Display the pretty table
print(df_pretty.to_latex(index=False))  # or .to_latex() if for LaTeX output

|    | Acquisition Function   | Iterations   | Final HV         | Pareto Spacing   | Mean HV Over Time   | Runtime (s)    |
|---:|:-----------------------|:-------------|:-----------------|:-----------------|:--------------------|:---------------|
|  0 | Sobol                  | 16.0 ± 5.61  | 429.38 ± 8.71    | 57.6 ± 64.56     | 422.39 ± 8.38       | 187.46 ± 22.59 |
|  1 | qParEGO                | 15.8 ± 5.93  | 430.47 ± 2.0     | 100.41 ± 97.46   | 426.74 ± 5.99       | 159.77 ± 17.27 |
|  2 | qQEHVI                 | 20.8 ± 10.69 | **432.4 ± 4.38** | 60.29 ± 49.05    | 429.6 ± 2.93        | 175.73 ± 19.85 |
|  3 | qQNEHVI                | 20.2 ± 6.98  | 431.85 ± 4.06    | 23.75 ± 8.77     | 430.38 ± 3.42       | 149.17 ± 5.96  |
\begin{tabular}{llllll}
\toprule
Acquisition Function & Iterations & Final HV & Pareto Spacing & Mean HV Over Time & Runtime (s) \\
\midrule
Sobol & 16.0 ± 5.61 & 429.38 ± 8.71 & 57.6 ± 64.56 & 422.39 ± 8.38 & 187.46 ± 22.59 \\
qParEGO & 15.8 ± 5.93 & 430.